In [1]:
# Load the data corresponding to each model
# Split train/val fold
# 

In [2]:
import sys
IN_COLAB = 'google.colab' in sys.modules

if IN_COLAB:

    # Enable line-wrapping
    # from IPython.display import HTML, display

    # def set_css():
    #     display(HTML('''
    #         <style>
    #         pre {
    #             white-space: pre-wrap;
    #             }
    #         </style>
    #     '''))
    # get_ipython().events.register('pre_run_cell', set_css)


    # mount google drive
    from google.colab import drive
    drive.mount('/content/drive')

    %pip install transformers
    %pip install datasets
    %pip install pynvml

In [3]:
if IN_COLAB:
    # Enable line-wrapping
    from IPython.display import HTML, display

    def set_css():
        display(HTML('''
            <style>
            pre {
                white-space: pre-wrap;
                }
            </style>
        '''))
    get_ipython().events.register('pre_run_cell', set_css)


    # mount google drive
    from google.colab import drive
    drive.mount('/content/drive')

In [4]:
import datetime
import numpy as np
from sklearn.metrics import fbeta_score
from matplotlib import pyplot as plt
import math
import pandas as pd
import sys
import json
import time
import torch
from torch import nn
import random
#from tqdm.auto import tqdm
from tqdm import tqdm
from torch.utils.data import DataLoader
from datasets import Dataset
import os
import torchsummary
from functools import partialmethod
from itertools import product

from transformers import AutoTokenizer, AutoModel
from dataclasses import dataclass
from transformers import PreTrainedTokenizer
from torch.utils.data import DataLoader

if IN_COLAB:
  sys.path.append('/content/drive/MyDrive/ML/lecr/code/')
from l_utils import *
from model_utils import *

#from model_defs import Topic_NN, Content_NN

%env CUBLAS_WORKSPACE_CONFIG=:16:8

device = "cuda" if torch.cuda.is_available() else "cpu"
print(device)

/home/mahesh/Desktop/ML/kaggle/LearningEquality/learning-equality-code/le-venv/lib/python3.10/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


2023-04-13 12:44:45.384476: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-04-13 12:44:45.505357: E tensorflow/stream_executor/cuda/cuda_blas.cc:2981] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


2023-04-13 12:44:45.965456: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/local/cuda-11.7/lib64:/usr/local/cuda-11.7/lib64
2023-04-13 12:44:45.965519: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer_plugin.so.7'; dlerror: libnvinfer_plugin.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/local/cuda-11.7/lib64:/usr/local/cuda-11.7/lib64
2023-04-13 12:44:45.965525: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Cannot dlopen some TensorRT libraries. If you would like to use Nvidia GPU with TensorRT, please make sure the missing libraries mentioned above are installed properly.


env: CUBLAS_WORKSPACE_CONFIG=:16:8
cuda


In [5]:
if IN_COLAB:
    class CFG:
        DIRNAME_PROCESSED_DATA = '/content/drive/MyDrive/ML/lecr/data/'
        FILENAME_DATA_CONTENT_F = DIRNAME_PROCESSED_DATA + "content.f"
        FILENAME_DATA_TOPICS_F = DIRNAME_PROCESSED_DATA + "topics.f"
        FILENAME_DATA_CORRELATIONS_CSV = DIRNAME_PROCESSED_DATA + "correlations.csv"
        NUM_FOLDS = 5
        SAVE_MODEL_DIR = '/content/drive/MyDrive/ML/lecr/models/'
else:
    from cfg import *

topics_tokenized_minilm_pq_file   = CFG.DIRNAME_PROCESSED_DATA + "topics_tokenized_ml64_new_sep_unique_id_minilm_mpnet.pq"
contents_tokenized_minilm_pq_file = CFG.DIRNAME_PROCESSED_DATA + "contents_tokenized_ml64_new_sep_unique_id_minilm_mpnet.pq"

topics_tokenized_dist_pq_file   = CFG.DIRNAME_PROCESSED_DATA + "topics_tokenized_ml64_dist.pq"
contents_tokenized_dist_pq_file = CFG.DIRNAME_PROCESSED_DATA + "contents_tokenized_ml64_dist.pq"


TRAIN_CORR_IDS_F        = CFG.DIRNAME_PROCESSED_DATA + "ranker_train_corr_ids.f"
VAL_PIPELINE_CORR_IDS_F = CFG.DIRNAME_PROCESSED_DATA + "ranker_val_pipeline_corr_ids.f"
VAL_CORR_IDS_F          = CFG.DIRNAME_PROCESSED_DATA + "ranker_val_corr_ids.f"

INFERENCE_BATCH_SIZE = 2000

In [6]:
class TransModel(torch.nn.Module):
    """
    Wrapper. Gets tokenized features and returns the output logits for whether the token and content are matching.
    """
    def __init__(self, trans_model_name):
        super().__init__()
        self.transformer = AutoModel.from_pretrained(trans_model_name)
    
    def get_transformer_output(self, input_ids, attention_mask):
        outputs = self.transformer(input_ids = input_ids, attention_mask=attention_mask)
        pooled_embeddings=mean_pooling_with_token_embs(outputs[0], attention_mask)
        return pooled_embeddings

    def forward(self, features):
        # Grab embeddings from transformer
        embs = self.get_transformer_output(
            features[:,0,:],
            features[:,1,:]
        )
        return embs


In [7]:
TRANS_MODEL_NAMES = ["sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2",
                     "sentence-transformers/paraphrase-multilingual-mpnet-base-v2",
                     "sentence-transformers/distiluse-base-multilingual-cased-v2"
                     ]
model_path = "/home/mahesh/Desktop/ML/kaggle/LearningEquality/models/from_colab/"

model_filenames = ['One_trans_minilm_MNRL_fp16_ml64_train_shuffled_parent_title_new_sep_unique_id_batch_size_1100_epoch_20_recall_10_0.793.pth',
                   'one_trans_mpnet_ArcFace_0.4_focal_loss_2.0_fp16_ml64_train_shuffled_parent_title_new_sep_unique_id_batch_size_1050_epoch_20_recall_10_0.796.pth',
                   'One_trans_distiluse_MNRL_fp16_ml64_train_shuffled_parent_title_new_sep_batch_size_1200_epoch_27_recall_10_0.787.pth'
                   ]


In [8]:
contents_df = pd.read_feather(CFG.FILENAME_DATA_CONTENT_F)
topics_df   = pd.read_feather(CFG.FILENAME_DATA_TOPICS_F)
correlations_df =  pd.read_csv(CFG.FILENAME_DATA_CORRELATIONS_CSV)

# MiniLM, mpnet tokenized
topics_tokenized_minilm_df   = pd.read_parquet(topics_tokenized_minilm_pq_file)
contents_tokenized_minilm_df = pd.read_parquet(contents_tokenized_minilm_pq_file)

# Distiluse tokenized
topics_tokenized_dist_df   = pd.read_parquet(topics_tokenized_dist_pq_file)
contents_tokenized_dist_df = pd.read_parquet(contents_tokenized_dist_pq_file)

train_df, val_df = create_train_val_split_pos_corr_from_kaggle(topics_df, correlations_df, num_splits = 4, val_fold = 0)

177290
102629
2023-04-13 12:44:49.747884 : Splitting non-source into 4 folds
fold
0.0    25667
1.0    25677
2.0    25663
3.0    25622
dtype: int64
fold
0.0    6070
1.0    4971
2.0    8493
3.0    5469
dtype: int64


In [9]:
val_topics_tokenized_minilm_df, val_contents_tokenized_minilm_df, val_corr_df = get_pos_corr_subsets_for_binary_fold( val_df, contents_tokenized_minilm_df, topics_tokenized_minilm_df, val = True)
val_topics_tokenized_dist_df, val_contents_tokenized_dist_df, val_corr_df     = get_pos_corr_subsets_for_binary_fold( val_df, contents_tokenized_dist_df, topics_tokenized_dist_df, val = True)

t_cols = [col for col in val_topics_tokenized_minilm_df.columns if col.startswith('t_')]
c_cols = [col for col in val_contents_tokenized_minilm_df.columns if col.startswith('c_')]
val_data_minilm = (torch.tensor(np.array(val_topics_tokenized_minilm_df[t_cols].values.tolist())),
                   torch.tensor(np.array(val_contents_tokenized_minilm_df[c_cols].values.tolist()))
                   ,val_corr_df
                   )

val_data_dist = (torch.tensor(np.array(val_topics_tokenized_dist_df[t_cols].values.tolist())),
                 torch.tensor(np.array(val_contents_tokenized_dist_df[c_cols].values.tolist()))
                 ,val_corr_df
                 )

In [10]:
val_topics_lang_minilm_df   = pd.merge(topics_df[['id', 'language']], val_topics_tokenized_minilm_df['id'])
val_topics_lang_dist_df     = pd.merge(topics_df[['id', 'language']], val_topics_tokenized_dist_df['id'])
val_contents_lang_minilm_df = pd.merge(contents_df[['id', 'language']], val_contents_tokenized_minilm_df['id'])
val_contents_lang_dist_df   = pd.merge(contents_df[['id', 'language']], val_contents_tokenized_dist_df['id'])

In [11]:
#
# Lets get the recall for unified retriever for the val data
#
CALCULATE_VAL_RECALL = True
if CALCULATE_VAL_RECALL:
    topics_features_l = [val_data_minilm[0], val_data_minilm[0], val_data_dist[0]]
    contents_features_l = [val_data_minilm[1], val_data_minilm[1], val_data_dist[1]]

    topics_df_l   = [val_topics_lang_minilm_df, val_topics_lang_minilm_df, val_topics_lang_dist_df]
    contents_df_l = [val_contents_lang_minilm_df, val_contents_lang_minilm_df, val_contents_lang_dist_df]

    # k_vals = [5,10, 50]
    k_vals = [10]

    val_matched_content_idxs_list = get_top_k_preds_for_models( trans_model_names = TRANS_MODEL_NAMES,
                                                                model_path = model_path,
                                                                model_filenames = model_filenames,
                                                                TransModel = TransModel,
                                                                device = device,
                                                                topics_features_l = topics_features_l,
                                                                contents_features_l = contents_features_l,
                                                                k_vals = k_vals,
                                                                batch_size = INFERENCE_BATCH_SIZE,
                                                                corr_data = val_data_minilm[2],
                                                                topics_df_l = topics_df_l,
                                                                contents_df_l = contents_df_l
                                                               )
    recalls = calculate_mean_average_recall_at_k(val_matched_content_idxs_list, k_vals, corr_data = val_data_minilm[2])
    print(f'\nUnified Mean Average Recalls: {recalls}')

2023-04-13 12:44:50.783627 : Loading model One_trans_minilm_MNRL_fp16_ml64_train_shuffled_parent_title_new_sep_unique_id_batch_size_1100_epoch_20_recall_10_0.793.pth ... 


2023-04-13 12:44:54.432314 : Computing content embeddings ...


  0% 0/78 [00:00<?, ?it/s]

  1% 1/78 [00:00<00:15,  4.91it/s]

  6% 5/78 [00:01<00:21,  3.41it/s]

...

100% 6069/6070 [01:57<00:00, 58.14it/s]

100% 6070/6070 [01:57<00:00, 51.65it/s]

(6070, 10)
2023-04-13 13:03:06.702940 : Garbage collection ... 
GPU:0
process     387645 uses     9803.000 MB GPU memory
GPU:0
process     387645 uses      699.000 MB GPU memory



Model Mean Average Recalls: {10: 0.738}



Unified Mean Average Recalls: {10: 0.841}


In [14]:
val_pipeline_corr_ids_df = get_corr_labeled_df(val_topics_tokenized_dist_df, val_contents_tokenized_dist_df, val_corr_df, val_matched_content_idxs_list[0])
val_corr_ids_df = get_corr_labeled_df(val_topics_tokenized_dist_df, val_contents_tokenized_dist_df, val_corr_df, val_matched_content_idxs_list[0], add_all_pos = True)
# No language filtering: 
# 2023-04-01 18:52:52.366769 : Num positive corrs: 25667
# 2023-04-01 18:52:52.558170 : Num pred corrs: 6070
# 2023-04-01 18:52:52.576164 : Num correct pred corrs: 21637
# 2023-04-01 18:52:52.624173 : Num positive corrs: 25667
# 2023-04-01 18:52:52.824444 : Num pred corrs: 6070
# 2023-04-01 18:52:52.844383 : Num correct pred corrs: 21637

2023-04-13 13:05:05.020932 : Num positive corrs: 25667
2023-04-13 13:05:05.224524 : Num pred corrs: 6070


2023-04-13 13:05:05.245713 : Num correct pred corrs: 20828
2023-04-13 13:05:05.310355 : Num positive corrs: 25667


2023-04-13 13:05:05.523821 : Num pred corrs: 6070
2023-04-13 13:05:05.543890 : Num correct pred corrs: 20828


In [ ]:
val_pipeline_corr_ids_df.info()
val_corr_ids_df.info()

In [15]:
val_pipeline_corr_ids_df.to_feather(VAL_PIPELINE_CORR_IDS_F)
val_corr_ids_df.to_feather(VAL_CORR_IDS_F)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 113372 entries, 0 to 113371
Data columns (total 3 columns):
 #   Column  Non-Null Count   Dtype 
---  ------  --------------   ----- 
 0   t_id    113372 non-null  object
 1   c_id    113372 non-null  object
 2   label   113372 non-null  int64 
dtypes: int64(1), object(2)
memory usage: 2.6+ MB
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 118211 entries, 0 to 118210
Data columns (total 3 columns):
 #   Column  Non-Null Count   Dtype 
---  ------  --------------   ----- 
 0   t_id    118211 non-null  object
 1   c_id    118211 non-null  object
 2   label   118211 non-null  int64 
dtypes: int64(1), object(2)
memory usage: 2.7+ MB


In [16]:
train_topics_tokenized_minilm_df, train_contents_tokenized_minilm_df, train_corr_df = get_pos_corr_subsets_for_binary_fold( train_df, contents_tokenized_minilm_df, topics_tokenized_minilm_df)
train_topics_tokenized_dist_df, train_contents_tokenized_dist_df, train_corr_df = get_pos_corr_subsets_for_binary_fold( train_df, contents_tokenized_dist_df, topics_tokenized_dist_df)

t_cols = [col for col in train_topics_tokenized_minilm_df.columns if col.startswith('t_')]
c_cols = [col for col in train_contents_tokenized_minilm_df.columns if col.startswith('c_')]
train_data_minilm = (torch.tensor(np.array(train_topics_tokenized_minilm_df[t_cols].values.tolist())),
                   torch.tensor(np.array(train_contents_tokenized_minilm_df[c_cols].values.tolist()))
                   ,train_corr_df
                   )

train_data_dist = (torch.tensor(np.array(train_topics_tokenized_dist_df[t_cols].values.tolist())),
                 torch.tensor(np.array(train_contents_tokenized_dist_df[c_cols].values.tolist()))
                 ,train_corr_df
                 )

In [17]:
train_topics_lang_minilm_df   = pd.merge(topics_df[['id', 'language']], train_topics_tokenized_minilm_df['id'])
train_topics_lang_dist_df     = pd.merge(topics_df[['id', 'language']], train_topics_tokenized_dist_df['id'])
train_contents_lang_minilm_df = pd.merge(contents_df[['id', 'language']], train_contents_tokenized_minilm_df['id'])
train_contents_lang_dist_df   = pd.merge(contents_df[['id', 'language']], train_contents_tokenized_dist_df['id'])

In [19]:
# Let's get the unified prediction for training data, we'll use k = 10
topics_features_l = [train_data_minilm[0], train_data_minilm[0], train_data_dist[0]]
contents_features_l = [train_data_minilm[1], train_data_minilm[1], train_data_dist[1]]

topics_df_l   = [train_topics_lang_minilm_df, train_topics_lang_minilm_df, train_topics_lang_dist_df]
contents_df_l = [train_contents_lang_minilm_df, train_contents_lang_minilm_df, train_contents_lang_dist_df]

k_vals = [10]

train_matched_content_idxs_list = get_top_k_preds_for_models( trans_model_names = TRANS_MODEL_NAMES,
                                                        model_path = model_path,
                                                        model_filenames = model_filenames,
                                                        TransModel = TransModel,
                                                        device = device,
                                                        topics_features_l = topics_features_l,
                                                        contents_features_l = contents_features_l,
                                                        k_vals = k_vals,
                                                        batch_size = INFERENCE_BATCH_SIZE,
                                                        corr_data = train_data_minilm[2],
                                                        topics_df_l = topics_df_l,
                                                        contents_df_l = contents_df_l
                                                       )
recalls = calculate_mean_average_recall_at_k(train_matched_content_idxs_list, k_vals, corr_data = train_data_minilm[2])
print(f'\nMean Average Recalls: {recalls}')
# Recalls without langugae filtering
# Model Recalls: {10: 0.936}
# Model Recalls: {10: 0.954}
# Model Recalls: {10: 0.94}
# Recalls: {10: 0.985}

2023-04-11 14:16:47.332041 : Loading model One_trans_minilm_MNRL_fp16_ml64_train_shuffled_parent_title_new_sep_unique_id_batch_size_1100_epoch_20_recall_10_0.793.pth ... 


2023-04-11 14:16:49.152561 : Computing content embeddings ...


  0% 0/75 [00:00<?, ?it/s]

  7% 5/75 [00:01<00:17,  3.98it/s]

  8% 6/75 [00:02<00:32,  2.13it/s]

...

100% 55447/55447 [11:27<00:00, 80.59it/s]

(55447, 10)
2023-04-11 15:05:27.526977 : Garbage collection ... 
GPU:0
process     220596 uses     9805.000 MB GPU memory


GPU:0
process     220596 uses      699.000 MB GPU memory



Model Mean Average Recalls: {10: 0.935}



Mean Average Recalls: {10: 0.978}


In [27]:
# Training Data:
# For training we need to extract retrieved results as (t_id, c_id) predictions. From there we need to label data with 0/1 labels. We need to add extra positive correlation data that was missed by the retreiver. From here we can augument the data necessary for training the ranker.

# Validation Data:
# For val data we need to extract retreived results as (t_id, c_id) predictions. From there we need to label data with 0/1 labels. From here we can augment the data necessary for validating the ranker. Is there a better way to validate the ranker? Can we directly use the correlation data from the original pos-correlations for the val set?



In [20]:
train_corr_ids_df = get_corr_labeled_df(train_topics_tokenized_dist_df, train_contents_tokenized_dist_df, train_corr_df, train_matched_content_idxs_list[0], add_all_pos = True)

2023-04-11 15:11:07.450233 : Num positive corrs: 254252


2023-04-11 15:11:09.484790 : Num pred corrs: 55447


2023-04-11 15:11:09.689016 : Num correct pred corrs: 229539


In [21]:
train_corr_ids_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 984357 entries, 0 to 984356
Data columns (total 3 columns):
 #   Column  Non-Null Count   Dtype 
---  ------  --------------   ----- 
 0   t_id    984357 non-null  object
 1   c_id    984357 non-null  object
 2   label   984357 non-null  int64 
dtypes: int64(1), object(2)
memory usage: 22.5+ MB


In [22]:
train_corr_ids_df.to_feather(TRAIN_CORR_IDS_F)

In [23]:
train_corr_ids_f = pd.read_feather(TRAIN_CORR_IDS_F)

In [24]:
pos = (train_corr_ids_f['label']==1).sum()
neg = (train_corr_ids_f['label']==0).sum()
pos/neg

0.34824032159757845

2023-04-11 15:11:32.975756 : Num positive corrs: 25667
2023-04-11 15:11:33.162037 : Num pred corrs: 6070
2023-04-11 15:11:33.179481 : Num correct pred corrs: 21669


2023-04-11 15:11:33.233134 : Num positive corrs: 25667
2023-04-11 15:11:33.413386 : Num pred corrs: 6070
2023-04-11 15:11:33.430572 : Num correct pred corrs: 21669


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 108930 entries, 0 to 108929
Data columns (total 3 columns):
 #   Column  Non-Null Count   Dtype 
---  ------  --------------   ----- 
 0   t_id    108930 non-null  object
 1   c_id    108930 non-null  object
 2   label   108930 non-null  int64 
dtypes: int64(1), object(2)
memory usage: 2.5+ MB
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 112928 entries, 0 to 112927
Data columns (total 3 columns):
 #   Column  Non-Null Count   Dtype 
---  ------  --------------   ----- 
 0   t_id    112928 non-null  object
 1   c_id    112928 non-null  object
 2   label   112928 non-null  int64 
dtypes: int64(1), object(2)
memory usage: 2.6+ MB
